In [2]:
cd ../

/Users/linafaik/Documents/projects/time-series-forecasting-models


/Users/linafaik/Documents/projects/time-series-forecasting-models/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import numpy as np
import os

from config import *
from src.metrics import *
from src.viz import *

%load_ext autoreload
%autoreload 2

## TimeLLM

In [ ]:
scenario_names = ['stat_models', 'dl_models', 'timellm_gpt2']
columns_additional = [
    'state_id', 'sold_quantity', 'sold_amount',
    'event_type_1', 'event_type_2', 'event_sporting', 'event_cultural',
    'event_national', 'event_religious'
    ]
columns_key = [time_col, id_col]

for i, name in enumerate(scenario_names):
    path = os.path.join("output",name, f"forecasts_{name}.csv")
    forecasts_sc_df = pd.read_csv(path)
    
    if i == 0:
        forecasts_df = forecasts_sc_df
    else:
        forecasts_df = forecasts_df.merge(
            forecasts_sc_df.drop(columns_additional, axis=1),
            on=columns_key
        )
        
forecasts_df.tail()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2,event_sporting,event_cultural,event_national,...,dow,month,dow_sin,dow_cos,month_sin,month_cos,is_weekend,TimeLLM,TimeLLM-lo-90,TimeLLM-hi-90
19405,2016-05-18,WI_3,WI,3268,9163.29,NaN,NaN,0,0,0,...,2,5,0.781831,0.623490,0.866025,-0.5,0,11118.027344,8336.855230,13899.199457
19406,2016-05-19,WI_3,WI,3398,9660.13,NaN,NaN,0,0,0,...,3,5,0.974928,-0.222521,0.866025,-0.5,0,11195.871094,8425.538949,13966.203238
19407,2016-05-20,WI_3,WI,4126,11982.37,NaN,NaN,0,0,0,...,4,5,0.433884,-0.900969,0.866025,-0.5,0,10994.149414,7840.379877,14147.918951
19408,2016-05-21,WI_3,WI,4519,12370.23,NaN,NaN,0,0,0,...,5,5,-0.433884,-0.900969,0.866025,-0.5,1,10766.265625,7707.408861,13825.122389
19409,2016-05-22,WI_3,WI,4757,13432.85,NaN,NaN,0,0,0,...,6,5,-0.974928,-0.222521,0.866025,-0.5,1,11099.627930,8267.374770,13931.881090


In [ ]:
# Initialize two containers:
# - `scores`: to store global evaluation metrics per model
# - `scores_per_ts`: to store evaluation metrics for each time series (by ID) per model
scores = {}
scores_per_ts = []

# Loop through all forecast model columns to evaluate
for column in ['AutoARIMA', 'AutoETS', 'CES', 'MSTL', 'Theta', 'NHITS-median', 'NBEATS-median', 'TimeLLM']:
    
    # Keep only rows where forecasts from the current model are available (i.e., not NaN)
    forecasts_filtered_df = forecasts_df[forecasts_df[column].notna()]
    
    # Compute overall metrics (e.g., MAE, RMSE, sMAPE) between actual values and model forecasts
    scores[column] = evaluate(
        forecasts_filtered_df[target_col],     # Ground truth
        forecasts_filtered_df[column]          # Forecasted values from the current model
    )
    
    # Compute metrics for each individual time series (grouped by id_col)
    scores_per_ts_model_df = (
        forecasts_filtered_df
        .groupby(id_col)
        .apply(lambda group: pd.Series(
            evaluate(group[target_col], group[column])   # Evaluate per time series
        ))
        .reset_index()
    )
    
    # Add model name to identify which model each row of metrics corresponds to
    scores_per_ts_model_df["model"] = column
    
    # Select columns in the right order and append to the list of all per-series scores
    scores_per_ts.append(
        scores_per_ts_model_df[["model"] + [c for c in scores_per_ts_model_df.columns if c != "model"]]
    )

# Convert the global metrics dictionary to a DataFrame with one row per model
scores_df = (
    pd.DataFrame(scores).T
    .reset_index()
    .rename(columns={"index": "model"})
)

# Concatenate the list of per-time-series score DataFrames into a single DataFrame
scores_per_ts_df = pd.concat(scores_per_ts, axis=0).reset_index(drop=True)


/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_13067/2857509405.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.Series(
/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_13067/2857509405.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.Series(
/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_13067/2857509405.py:23: Depre

In [ ]:
import plotly.express as px

# Configuration
metric = "MAPE"
as_percentage = True  # Set to True if the metric should be displayed as %

# Add model type mapping
model_types = {
    "AutoARIMA": "Statistical",
    "AutoETS": "Statistical",
    "CES": "Statistical",
    "MSTL": "Statistical",
    "Theta": "Statistical",
    "NHITS-median": "Deep Learning",
    "NBEATS-median": "Deep Learning",
    "TimeLLM": "TimeLLM"
}
scores_df["model_type"] = scores_df["model"].map(model_types)

# Convert metric to percentage if needed
if as_percentage:
    scores_df[f"{metric}_perc"] = scores_df[metric] * 100
    y_axis_label = f"{metric} (%)"
    text_template = '%{text:.1f}%'
else:
    y_axis_label = metric
    text_template = '%{text:.2f}'

# Define color palette
color_map = {
    "Statistical": "#8ACCD5",
    "Deep Learning": "#C1DBB3",
    "TimeLLM": "#FE7743"
}

# Create the plot
fig = px.bar(
    scores_df,
    x="model",
    y=f"{metric}_perc",
    color="model_type",
    color_discrete_map=color_map,
    title=f"Model Comparison by {metric}",
    text=f"{metric}_perc",
)
fig.update_traces(texttemplate=text_template, textposition='outside')
fig.update_layout(
    yaxis_range=[0, 1.2 * scores_df[f"{metric}_perc"].max()],
    yaxis_title=y_axis_label, 
    xaxis_title="Model", 
    legend_title="Model Type",
    height=400,
    width=800
    )

fig.show()

In [ ]:
# Add model type mapping
model_types = {
    "AutoARIMA": "Statistical",
    "AutoETS": "Statistical",
    "CES": "Statistical",
    "MSTL": "Statistical",
    "Theta": "Statistical",
    "NHITS-median": "Deep Learning",
    "NBEATS-median": "Deep Learning",
    "TimeLLM": "TimeLLM"
}
scores_per_ts_df["model_type"] = scores_per_ts_df["model"].map(model_types)

# Convert metric to percentage if needed
if as_percentage:
    scores_per_ts_df[f"{metric}_perc"] = scores_per_ts_df[metric] * 100
    y_axis_label = f"{metric} (%)"
else:
    scores_per_ts_df[f"{metric}_perc"] = scores_per_ts_df[metric]
    y_axis_label = metric

# Define color palette
color_map = {
    "Statistical": "#8ACCD5",
    "Deep Learning": "#C1DBB3",
    "TimeLLM": "#FE7743"
}

# Create boxplot
fig = px.box(
    scores_per_ts_df,
    x="model",
    y=f"{metric}_perc",
    color="model_type",
    color_discrete_map=color_map,
    points="all",
    title=f"Distribution of {metric} by Model across Stores"
)

fig.update_layout(
    yaxis_title=y_axis_label, 
    xaxis_title="Model", 
    legend_title="Model Type",
    height=450,
    width=850
)

fig.show()


In [7]:
rnd_ids = scores_per_ts_df[id_col].unique()

map_columns_forecasts = {
    #'AutoARIMA': 'AutoARIMA',
    'AutoETS': 'AutoETS',
    #'CES': 'CES',
    #'MSTL': 'MSTL',
    #'Theta': 'Theta',
    #'NHITS': 'NHITS-median',
    'NBEATS': 'NBEATS-median',
    'TimeLLM': 'TimeLLM'
    }

for uid in rnd_ids:
    
    fig = plot_forecasts_uid(
        uid=uid,
        forecasts_df=forecasts_df, 
        column_id=id_col,
        column_date=time_col,
        column_target=target_col,
        map_columns_forecasts=map_columns_forecasts, 
        scores_per_ts_df=scores_per_ts_df, 
        metric="MAPE", 
        train_tail=100,
        map_color_forecasts={column_forecast: color_map[model_types[column_forecast]] for column_forecast in map_columns_forecasts.values()},
        as_percentage=True
    )
    
    fig.show()


/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:116: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-

In [12]:
rnd_id = np.random.choice(scores_per_ts_df[id_col].unique())

for model_name, column_name in map_columns_forecasts.items():

    fig = plot_forecast_with_ci(
        forecasts_df, 
        column_id=id_col,
        column_date=time_col,
        column_target=target_col,
        column_forecast=column_name,
        uid=rnd_id, 
        model_name=model_name,
        level=90, 
        train_tail=30
        )
    
    fig.update_layout(title=f"Forecast with 90% CI for {model_name} on ID {rnd_id}")
    fig.show()


/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:183: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:185: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:183: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:185: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:183: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:185: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



## TimeGPT

In [6]:
scenario_names = ['stat_models', 'dl_models']
scenario_timegpt_names = ['timegpt_baseline', 'timegpt_with_date_feat', 'timegpt_long_horizon', 'timegpt_with_exog_feat']
columns_additional = [
    'state_id', 'sold_quantity', 'sold_amount',
    'event_type_1', 'event_type_2', 'event_sporting', 'event_cultural',
    'event_national', 'event_religious'
    ]
columns_key = [time_col, id_col]

for i, name in enumerate(scenario_names):
    path = os.path.join("output",name, f"forecasts_{name}.csv")
    forecasts_sc_df = pd.read_csv(path)
    
    if i == 0:
        forecasts_df = forecasts_sc_df
    else:
        forecasts_df = forecasts_df.merge(
            forecasts_sc_df.drop(columns_additional, axis=1),
            on=columns_key
        )

for i, name in enumerate(scenario_timegpt_names):
    path = os.path.join("output",name, f"forecasts_{name}.csv")
    formatted_name = f'TimeGPT ({name.replace("timegpt_", "").replace("_", " ")})'
    
    forecasts_sc_df = pd.read_csv(path).rename(columns={"TimeGPT": formatted_name})
    forecasts_df = forecasts_df.merge(
        forecasts_sc_df.drop(columns_additional, axis=1),
        on=columns_key
    )
        
forecasts_df.tail()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2,event_sporting,event_cultural,event_national,...,NHITS-median,NHITS-lo-90,NHITS-hi-90,NBEATS-median,NBEATS-lo-90,NBEATS-hi-90,TimeGPT (baseline),TimeGPT (with date feat),TimeGPT (long horizon),TimeGPT (with exog feat)
19405,2016-05-18,WI_3,WI,3268,9163.29,NaN,NaN,0,0,0,...,9388.312,8049.8022,10411.233,8962.190,7848.9297,9986.906,9686.321,9450.850,9297.760,9562.100
19406,2016-05-19,WI_3,WI,3398,9660.13,NaN,NaN,0,0,0,...,9280.735,8048.7760,10383.619,9247.131,8227.1980,10229.908,9896.385,9557.519,9525.980,9743.397
19407,2016-05-20,WI_3,WI,4126,11982.37,NaN,NaN,0,0,0,...,10612.061,9182.0670,11701.540,10976.120,9879.1000,12282.996,11178.528,10900.216,11045.677,11169.068
19408,2016-05-21,WI_3,WI,4519,12370.23,NaN,NaN,0,0,0,...,12475.693,11032.9760,14058.324,12602.628,11297.1780,14294.606,13649.355,13038.910,13141.402,13329.268
19409,2016-05-22,WI_3,WI,4757,13432.85,NaN,NaN,0,0,0,...,12222.669,10595.5100,13889.405,12591.581,10886.4370,14096.614,13287.627,12892.754,12514.386,12962.116


In [13]:
# Initialize two containers:
# - `scores`: to store global evaluation metrics per model
# - `scores_per_ts`: to store evaluation metrics for each time series (by ID) per model
scores = {}
scores_per_ts = []
scenario_timegpt_formatted_names = [f"TimeGPT ({name.replace('timegpt_', '').replace('_', ' ')})" for name in scenario_timegpt_names]

# Loop through all forecast model columns to evaluate
for column in ['AutoARIMA', 'AutoETS', 'CES', 'MSTL', 'Theta', 'NHITS-median', 'NBEATS-median']+ scenario_timegpt_formatted_names:
    
    # Keep only rows where forecasts from the current model are available (i.e., not NaN)
    forecasts_filtered_df = forecasts_df[forecasts_df[column].notna()]
    
    # Compute overall metrics (e.g., MAE, RMSE, sMAPE) between actual values and model forecasts
    scores[column] = evaluate(
        forecasts_filtered_df[target_col],     # Ground truth
        forecasts_filtered_df[column]          # Forecasted values from the current model
    )
    
    # Compute metrics for each individual time series (grouped by id_col)
    scores_per_ts_model_df = (
        forecasts_filtered_df
        .groupby(id_col)
        .apply(lambda group: pd.Series(
            evaluate(group[target_col], group[column])   # Evaluate per time series
        ))
        .reset_index()
    )
    
    # Add model name to identify which model each row of metrics corresponds to
    scores_per_ts_model_df["model"] = column
    
    # Select columns in the right order and append to the list of all per-series scores
    scores_per_ts.append(
        scores_per_ts_model_df[["model"] + [c for c in scores_per_ts_model_df.columns if c != "model"]]
    )

# Convert the global metrics dictionary to a DataFrame with one row per model
scores_df = (
    pd.DataFrame(scores).T
    .reset_index()
    .rename(columns={"index": "model"})
)

# Concatenate the list of per-time-series score DataFrames into a single DataFrame
scores_per_ts_df = pd.concat(scores_per_ts, axis=0).reset_index(drop=True)
scores_df

/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_95869/2075829188.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  forecasts_filtered_df
/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_95869/2075829188.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  forecasts_filtered_df
/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_95869/2075829188.py:22: FutureWarning: DataFrameGroupBy.app

,model,MAE,RMSE,MAPE,R2,count
0,AutoARIMA,1345.850419,2.915555e+06,0.091815,0.829123,140.0
1,AutoETS,1181.079193,2.331103e+06,0.080433,0.863377,140.0
2,CES,1252.440603,2.889545e+06,0.084590,0.830647,140.0
3,MSTL,1290.309465,2.858886e+06,0.087434,0.832444,140.0
4,Theta,1216.612549,2.508826e+06,0.081705,0.852961,140.0
5,NHITS-median,1163.569738,2.296756e+06,0.077986,0.865390,140.0
6,NBEATS-median,1058.553978,1.987458e+06,0.071381,0.883518,140.0
7,TimeGPT (baseline),1222.124056,2.541648e+06,0.081830,0.851037,140.0
8,TimeGPT (with date feat),1216.554253,2.699880e+06,0.080050,0.841763,140.0
9,TimeGPT (long horizon),1178.283678,2.525547e+06,0.078074,0.851981,140.0


In [16]:
import plotly.express as px

# Configuration
metric = "MAPE"
as_percentage = True  # Set to True if the metric should be displayed as %

# Add model type mapping
model_types = {
    "AutoARIMA": "Statistical",
    "AutoETS": "Statistical",
    "CES": "Statistical",
    "MSTL": "Statistical",
    "Theta": "Statistical",
    "NHITS-median": "Deep Learning",
    "NBEATS-median": "Deep Learning",
}
model_types = {**model_types, **{c:"TimeGPT" for c in scenario_timegpt_formatted_names}}
scores_df["model_type"] = scores_df["model"].map(model_types)

# Convert metric to percentage if needed
if as_percentage:
    scores_df[f"{metric}_perc"] = scores_df[metric] * 100
    y_axis_label = f"{metric} (%)"
    text_template = '%{text:.1f}%'
else:
    y_axis_label = metric
    text_template = '%{text:.2f}'

# Define color palette
color_map = {
    "Statistical": "#8ACCD5",
    "Deep Learning": "#C1DBB3",
    "TimeGPT": "#FE7743"
}

# Create the plot
fig = px.bar(
    scores_df,
    x="model",
    y=f"{metric}_perc",
    color="model_type",
    color_discrete_map=color_map,
    title=f"Model Comparison by {metric}",
    text=f"{metric}_perc",
)
fig.update_traces(texttemplate=text_template, textposition='outside')
fig.update_layout(
    yaxis_range=[0, 1.2 * scores_df[f"{metric}_perc"].max()],
    yaxis_title=y_axis_label, 
    xaxis_title="Model", 
    legend_title="Model Type",
    height=400,
    width=800
    )

fig.show()

In [18]:
scores_per_ts_df["model_type"] = scores_per_ts_df["model"].map(model_types)

# Convert metric to percentage if needed
if as_percentage:
    scores_per_ts_df[f"{metric}_perc"] = scores_per_ts_df[metric] * 100
    y_axis_label = f"{metric} (%)"
else:
    scores_per_ts_df[f"{metric}_perc"] = scores_per_ts_df[metric]
    y_axis_label = metric

# Create boxplot
fig = px.box(
    scores_per_ts_df,
    x="model",
    y=f"{metric}_perc",
    color="model_type",
    color_discrete_map=color_map,
    points="all",
    title=f"Distribution of {metric} by Model across Stores"
)

fig.update_layout(
    yaxis_title=y_axis_label, 
    xaxis_title="Model", 
    legend_title="Model Type",
    height=450,
    width=1000
)

fig.show()
